In [4]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
from bs4 import BeautifulSoup
import requests

import datetime
import pytz

bangkok_tz = pytz.timezone('Asia/Bangkok')



# Path to your service account JSON file
service_account_path = "./bims-432306-47663614f7ab.json"

# Load the service account credentials
credentials = service_account.Credentials.from_service_account_file(service_account_path)

# Initialize the BigQuery client
client = bigquery.Client(credentials=credentials, project=credentials.project_id)





In [3]:
# # Query Data


# # Define your SQL query
# query = """
#     SELECT *
#     FROM `bims-432306.BIMS_data.ACTW_*`
#     LIMIT 10
# """

# # Execute the query
# query_job = client.query(query)

# # Fetch the results
# results = query_job.result()

# # Process the results (print them out)
# for row in results:
#     print(dict(row))

In [ ]:
## download data from BIMS

In [8]:
tabname = 'ACTW'

date_start = '2024-07-01'
sharding_name = (datetime.datetime.strptime(date_start,"%Y-%m-%d")).strftime(format='%Y%m%d')
date_end = ((datetime.datetime.strptime(date_start,"%Y-%m-%d")) + datetime.timedelta(days=1)).strftime(format = "%Y-%m-%d")

table_id = "bims-432306.BIMS_data.ACTW_"+sharding_name

In [10]:
url = f'http://110.49.150.135:4002/CPU/?command=DataQuery&uri=dl:tab{tabname}&format=html&mode=date-range&p1={date_start}T00:00:00&p2={date_end}T00:00:00'

page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

table = soup.find('table')
rows = table.find_all('tr')
data = []
for row in rows:
    cols = row.find_all(['td', 'th'])
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

df = pd.DataFrame(data[1:],columns=data[0])

collist= [col for col in df.columns if col not in ['TimeStamp','Record','date']]

# df['TimeStamp'] = pd.to_datetime(df['TimeStamp']).dt.tz_localize(bangkok_tz)


for col in collist:
    df[col] = df[col].astype('float')



In [15]:
df.TimeStamp[0]

'2024-04-30 00:00:00.000'

In [10]:
df

,TimeStamp,Record,WaterTemp_ACTW,Conductivity,Salinity,ACTWVoltage
0,2024-07-01 00:00:00+07:00,211774,30.78,51.92,30.20,12.43
1,2024-07-01 00:01:00+07:00,211775,30.79,51.92,30.20,12.43
2,2024-07-01 00:02:00+07:00,211776,30.71,51.70,30.10,12.43
3,2024-07-01 00:03:00+07:00,211777,30.72,51.72,30.11,12.43
4,2024-07-01 00:04:00+07:00,211778,30.67,51.61,30.07,12.43
...,...,...,...,...,...,...
1435,2024-07-01 23:55:00+07:00,213209,30.99,51.50,29.80,12.51
1436,2024-07-01 23:56:00+07:00,213210,31.01,51.92,30.07,12.50
1437,2024-07-01 23:57:00+07:00,213211,31.01,52.11,30.18,12.50
1438,2024-07-01 23:58:00+07:00,213212,31.01,52.21,30.25,12.50


In [12]:
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Change to WRITE_TRUNCATE to overwrite
    autodetect=True,  # Automatically infer schema
)

# Upload the DataFrame to BigQuery
job = client.load_table_from_dataframe(
    df, 
    table_id, 
    job_config=job_config
)

# Wait for the job to complete
job.result()

print(f"Loaded {job.output_rows} rows into {table_id}.")

Loaded 1440 rows into bims-432306.BIMS_data.ACTW_20240701.


In [15]:
tabname = 'AROW'

# Start and end dates
start_date = datetime.datetime.strptime('2024-08-01', '%Y-%m-%d')
end_date = datetime.datetime.strptime('2024-08-02', '%Y-%m-%d')

# Create a list to hold the dates
date_list = []

# Iterate from start date to end date
current_date = start_date
while current_date <= end_date:
    date_list.append(current_date.strftime('%Y-%m-%d'))
    current_date += datetime.timedelta(days=1)

# Output the list
print(date_list)


['2024-08-01', '2024-08-02']


In [16]:
for mydate in date_list:

    ## prepare
    date_start = mydate
    sharding_name = (datetime.datetime.strptime(date_start,"%Y-%m-%d")).strftime(format='%Y%m%d')
    date_end = ((datetime.datetime.strptime(date_start,"%Y-%m-%d")) + datetime.timedelta(days=1)).strftime(format = "%Y-%m-%d")

    table_id = "bims-432306.BIMS_data."+tabname+"_"+sharding_name

    table_id_clean = "bims-432306.BIMS_data."+tabname+"_clean_"+sharding_name

    ## Download from BIMS
    url = f'http://110.49.150.135:4002/CPU/?command=DataQuery&uri=dl:tab{tabname}&format=html&mode=date-range&p1={date_start}T00:00:00&p2={date_end}T00:00:00'

    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    table = soup.find('table')
    rows = table.find_all('tr')
    data = []
    for row in rows:
        cols = row.find_all(['td', 'th'])
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])

    df = pd.DataFrame(data[1:],columns=data[0])

    collist= [col for col in df.columns if col not in ['TimeStamp','Record','date']]

    # df['TimeStamp'] = pd.to_datetime(df['TimeStamp']).dt.tz_localize(bangkok_tz)

    for col in collist:
        df[col] = df[col].astype('float')

    print('Finish downloading for ' + mydate)


    ### Create a new dataframe for cleaning data:

    df_clean = df.copy()
    if table_id == "ACTW":
        df_clean = df_clean[['TimeStamp','Record','WaterTemp_ACTW','Salinity']].copy()
    elif table_id == "ACLW":
        df_clean = df_clean[['TimeStamp','Record','Chlorophyll']].copy()
    elif table_id == "AROW":
        df_clean = df_clean[['TimeStamp','Record','DO_AROW']].copy()

    collist= [col for col in df_clean.columns if col not in ['TimeStamp','Record']]

    for col in collist:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)

        # Calculate IQR
        IQR = Q3 - Q1

        # Define outlier boundaries
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Detect outliers
        outliers = (df[col] < lower_bound) | (df[col] > upper_bound)

        # Remove outliers
        df_clean.loc[outliers,col] = float('nan')




    ## Load the whole thing to GBQ

    job_config = bigquery.LoadJobConfig(
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Change to WRITE_TRUNCATE to overwrite
        autodetect=True,  # Automatically infer schema
    )

    # Upload the DataFrame to BigQuery
    job = client.load_table_from_dataframe(
        df, 
        table_id, 
        job_config=job_config
    )

    # Wait for the job to complete
    job.result()
    print(f"Loaded {job.output_rows} rows into {table_id}.")



    ## Load the clean df to GBQ
    # Upload the DataFrame to BigQuery
    job = client.load_table_from_dataframe(
        df_clean, 
        table_id_clean, 
        job_config=job_config
    )

    # Wait for the job to complete
    job.result()

    print(f"Loaded {job.output_rows} rows into {table_id_clean}.")
    

    



    



Finish downloading for 2024-08-01
Loaded 1436 rows into bims-432306.BIMS_data.AROW_20240801.
Loaded 1436 rows into bims-432306.BIMS_data.AROW_clean_20240801.
Finish downloading for 2024-08-02
Loaded 1423 rows into bims-432306.BIMS_data.AROW_20240802.
Loaded 1423 rows into bims-432306.BIMS_data.AROW_clean_20240802.


In [8]:
collist

['LoggerVoltage_Min',
 'LoggerTemp',
 'WaterTemp_AROW',
 'DO_AROW',
 'AROWVoltage',
 'DO_mgL']

In [10]:
df.shape

(1423, 8)

In [13]:
df_clean.isna().sum()

TimeStamp              0
Record                 0
LoggerVoltage_Min      0
LoggerTemp             0
WaterTemp_AROW         0
DO_AROW               88
AROWVoltage            0
DO_mgL               102
dtype: int64

In [7]:
## prepare
date_start = '2024-08-17'
tabname='ACTW'
sharding_name = (datetime.datetime.strptime(date_start,"%Y-%m-%d")).strftime(format='%Y%m%d')
date_end = ((datetime.datetime.strptime(date_start,"%Y-%m-%d")) + datetime.timedelta(days=1)).strftime(format = "%Y-%m-%d")

table_id = "bims-432306.BIMS_data."+tabname+"_"+sharding_name

## Download from BIMS
url = f'http://110.49.150.135:4002/CPU/?command=DataQuery&uri=dl:tab{tabname}&format=html&mode=date-range&p1={date_start}T00:00:00&p2={date_end}T00:00:00'

page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

table = soup.find('table')
rows = table.find_all('tr')
data = []
for row in rows:
    cols = row.find_all(['td', 'th'])
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

df = pd.DataFrame(data[1:],columns=data[0])

collist= [col for col in df.columns if col not in ['TimeStamp','Record','date']]

# df['TimeStamp'] = pd.to_datetime(df['TimeStamp']).dt.tz_localize(bangkok_tz)

for col in collist:
    df[col] = df[col].astype('float')

print('Finish downloading for ' + date_start)


### Create a new dataframe for cleaning data:

df_clean = df.copy()
for col in collist:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)

    # Calculate IQR
    IQR = Q3 - Q1

    # Define outlier boundaries
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Detect outliers
    outliers = (df[col] < lower_bound) | (df[col] > upper_bound)

    # Remove outliers
    df_clean.loc[outliers,col] = float('nan')

Finish downloading for 2024-08-17


In [8]:
df

,TimeStamp,Record,WaterTemp_ACTW,Conductivity,Salinity,ACTWVoltage
0,2024-08-17 00:00:00.000,279403,31.13,49.69,28.55,12.50
1,2024-08-17 00:01:00.000,279404,31.11,49.57,28.48,12.50
2,2024-08-17 00:02:00.000,279405,31.15,49.69,28.54,12.50
3,2024-08-17 00:03:00.000,279406,31.10,49.45,28.42,12.50
4,2024-08-17 00:04:00.000,279407,31.09,49.44,28.41,12.50
...,...,...,...,...,...,...
1435,2024-08-17 23:55:00.000,280838,30.98,48.98,28.18,12.45
1436,2024-08-17 23:56:00.000,280839,30.98,48.87,28.12,12.45
1437,2024-08-17 23:57:00.000,280840,30.97,48.88,28.12,12.45
1438,2024-08-17 23:58:00.000,280841,30.95,48.87,28.13,12.45


In [10]:
df_clean.isna().sum()

TimeStamp           0
Record              0
WaterTemp_ACTW      0
Conductivity      174
Salinity          181
ACTWVoltage       106
dtype: int64